In [1]:
import os
import cv2
import numpy as np
from pathlib import Path
import copy
import time

In [2]:
# with b 16 and eps 1, the face is clear and the noise can be seen
# with b 16 and eps 0.5, there is still a face but more noise
# with b 16 and eps 0.1, there is no face

# for b = 8,16,32, the blocks are too small and hence the noise added is huge
# face started appearing for b = 40, 48, 56
# though the face was not that clear, we stopped at 64
# we wanted to keep the noise at its highest for this so did not change the eps


# tried with b 64, and a higher eps value but the face was very pixelated
# tried again with b 32 and eps 0.5 the face is relatively better
# tried again with b 24 and eps 0.5 and the face is much better



#An interesting observation is that as we have to pixelate the images first, for both DP Pix and DP Blur 
#even after the eps is increased so much i.e. to 100 or even even higher, and it practically nullifies the noise added
#by the laplace distribution, the original impact of the pixelation will still be there
#hence the block size chosen for pixelating the images has to be carefully selected
#the authors did not realize this, as their dataset only contained images of 92 * 112, and they did not experiment with
# a b size of more than 16, and for lower values of b, they concluded that increasing the size of b will result in higher
#utility, which is not a correct inference.

In [3]:
# Define the path to the input and output folders
input_dir = "F:/Original/All FR/train"
output_dir = "F:/Anonymized/Original/DP_Pix/b_8_eps_10/Face_Recognition"

b= 8
eps = 10
m = 16

In [4]:
# Create the output directory if it does not exist
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [5]:
def create_dp_pixelated_image(x, b, m, eps):
    img = torch.squeeze(x)
    img1 = np.transpose(img.cpu().detach().numpy(), (1, 2, 0))
    
    
    pix = Pix(img1, b)
    DP_Pixed = DP_Pix(pix, b, m, eps)
    
    DP_Pixed1 = torch.from_numpy(np.transpose(DP_Pixed, (2, 0, 1)))
    DP_Pixed_tensor = torch.unsqueeze(DP_Pixed1, 0)
    
    return DP_Pixed_tensor

In [6]:
def Pix(X_arr, b):
    X_noise = []
    
    (b_channel, g_channel, r_channel) = cv2.split(X_arr)
    new_b = copy.copy(b_channel)
    new_g = copy.copy(g_channel)
    new_r = copy.copy(r_channel)
    x, y = new_b.shape
    
    for i in range(0, x, b):
        for j in range(0, y, b):
            new_b[i:i + b, j:j + b] = new_b[i:i + b, j:j + b].mean(axis=(0, 1))
    for i in range(0, x, b):
        for j in range(0, y, b):
            new_g[i:i + b, j:j + b] = new_g[i:i + b, j:j + b].mean(axis=(0, 1))
    for i in range(0, x, b):
        for j in range(0, y, b):
            new_r[i:i + b, j:j + b] = new_r[i:i + b, j:j + b].mean(axis=(0, 1))
                
    X_noise = cv2.merge((new_b,new_g,new_r))
    
    return X_noise

In [7]:
def DP_Pix(X_arr, b, m, eps):
    
    
    X_noise = []
    (b_channel, g_channel, r_channel) = cv2.split(X_arr)
    new_b = copy.copy(b_channel)
    new_g = copy.copy(g_channel)
    new_r = copy.copy(r_channel)
    x, y = new_b.shape
    
    for i in range(0, x, b):
        for j in range(0, y, b):
            new_b[i:i + b, j:j + b] = new_b[i:i + b, j:j + b].mean(axis=(0, 1)) \
                                       + np.random.laplace(loc=0.0, scale=(255 * m) / ((b ** 2) * eps), size=None)
    for i in range(0, x, b):
        for j in range(0, y, b):
            new_g[i:i + b, j:j + b] = new_g[i:i + b, j:j + b].mean(axis=(0, 1)) \
                                       + np.random.laplace(loc=0.0, scale=(255 * m) / ((b ** 2) * eps), size=None)
    for i in range(0, x, b):
        for j in range(0, y, b):
            new_r[i:i + b, j:j + b] = new_r[i:i + b, j:j + b].mean(axis=(0, 1)) \
                                           + np.random.laplace(loc=0.0, scale=(255 * m) / ((b ** 2) * eps), size=None)
                
    X_noise = cv2.merge((new_b,new_g,new_r))
    return X_noise

In [8]:
start_time = time.time()
# Loop through each folder in the input directory
for foldername in os.listdir(input_dir):
    folder_path = os.path.join(input_dir, foldername)
    if os.path.isdir(folder_path):
        # Create the corresponding subfolder in the output directory
        output_folder_path = os.path.join(output_dir, foldername)
        Path(output_folder_path).mkdir(parents=True, exist_ok=True)

        # Loop through each file in the folder
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                img = cv2.imread(file_path)
                if img is None:
                    print(f"Skipping {file_path}")
                    continue

                img = Pix(img, b)
                img = DP_Pix(img, b, m, eps)

                # Save the processed image to the output folder
                output_file_path = os.path.join(output_folder_path, filename)
                cv2.imwrite(output_file_path, img)
end_time = time.time()
overall_execution_time = end_time - start_time
print(f"\nOverall execution time: {overall_execution_time} seconds")
print("All images processed.")


Overall execution time: 3371.1568353176117 seconds
All images processed.


In [ ]:
Overall execution time: 902.8989746570587 seconds
All images processed.

In [ ]:
# Define the path to the input and output folders
# Define the path to the input and output folders
input_dir = "F:/Original/All FR/train"
output_dir = "F:/Anonymized/Original/DP_Pix/b_32_eps_0.5/Face_Recognition"

b= 32
eps = 0.5
m = 16

In [ ]:
# Create the output directory if it does not exist
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
start_time = time.time()
# Loop through each folder in the input directory
for foldername in os.listdir(input_dir):
    folder_path = os.path.join(input_dir, foldername)
    if os.path.isdir(folder_path):
        # Create the corresponding subfolder in the output directory
        output_folder_path = os.path.join(output_dir, foldername)
        Path(output_folder_path).mkdir(parents=True, exist_ok=True)

        # Loop through each file in the folder
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                img = cv2.imread(file_path)
                if img is None:
                    print(f"Skipping {file_path}")
                    continue

                img = Pix(img, b)
                img = DP_Pix(img, b, m, eps)

                # Save the processed image to the output folder
                output_file_path = os.path.join(output_folder_path, filename)
                cv2.imwrite(output_file_path, img)
end_time = time.time()
overall_execution_time = end_time - start_time
print(f"\nOverall execution time: {overall_execution_time} seconds")
print("All images processed.")

In [ ]:

Overall execution time: 939.7353801727295 seconds
All images processed.

In [ ]:
# Define the path to the input and output folders
# Define the path to the input and output folders
input_dir = "F:/Original/All FR/train"
output_dir = "F:/Anonymized/Original/DP_Pix/b_32_eps_1/Face_Recognition"

b= 32
eps = 1
m = 16

In [ ]:
# Create the output directory if it does not exist
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
start_time = time.time()
# Loop through each folder in the input directory
for foldername in os.listdir(input_dir):
    folder_path = os.path.join(input_dir, foldername)
    if os.path.isdir(folder_path):
        # Create the corresponding subfolder in the output directory
        output_folder_path = os.path.join(output_dir, foldername)
        Path(output_folder_path).mkdir(parents=True, exist_ok=True)

        # Loop through each file in the folder
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                img = cv2.imread(file_path)
                if img is None:
                    print(f"Skipping {file_path}")
                    continue

                img = Pix(img, b)
                img = DP_Pix(img, b, m, eps)

                # Save the processed image to the output folder
                output_file_path = os.path.join(output_folder_path, filename)
                cv2.imwrite(output_file_path, img)
end_time = time.time()
overall_execution_time = end_time - start_time
print(f"\nOverall execution time: {overall_execution_time} seconds")
print("All images processed.")